In [ ]:
# %%

from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.tabular.configs.hyperparameter_configs import get_hyperparameter_config

import numpy as np

from sklearnex import patch_sklearn
patch_sklearn()

label = 'signature'
time_limit = 3600 # seconds (1 hour)

data_url = 'https://raw.githubusercontent.com/mli/ag-docs/main/knot_theory/'

train_data = TabularDataset(f'{data_url}train.csv')
test_data = TabularDataset(f'{data_url}test.csv')

train_data.head()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


,Unnamed: 0,chern_simons,cusp_volume,hyperbolic_adjoint_torsion_degree,hyperbolic_torsion_degree,injectivity_radius,longitudinal_translation,meridinal_translation_imag,meridinal_translation_real,short_geodesic_imag_part,short_geodesic_real_part,Symmetry_0,Symmetry_D3,Symmetry_D4,Symmetry_D6,Symmetry_D8,Symmetry_Z/2 + Z/2,volume,signature
0,70746,0.090530,12.226322,0,10,0.507756,10.685555,1.144192,-0.519157,-2.760601,1.015512,0.0,0.0,0.0,0.0,0.0,1.0,11.393225,-2
1,240827,0.232453,13.800773,0,14,0.413645,10.453156,1.320249,-0.158522,-3.013258,0.827289,0.0,0.0,0.0,0.0,0.0,1.0,12.742782,0
2,155659,-0.144099,14.761030,0,14,0.436928,13.405199,1.101142,0.768894,2.233106,0.873856,0.0,0.0,0.0,0.0,0.0,0.0,15.236505,2
3,239963,-0.171668,13.738019,0,22,0.249481,27.819496,0.493827,-1.188718,-2.042771,0.498961,0.0,0.0,0.0,0.0,0.0,0.0,17.279890,-8
4,90504,0.235188,15.896359,0,10,0.389329,15.330971,1.036879,0.722828,-3.056138,0.778658,0.0,0.0,0.0,0.0,0.0,0.0,16.749298,4


In [ ]:
# %%

custom_hyperparameters = get_hyperparameter_config('default')

custom_hyperparameters['LR'] = [
    {'multi_class':'multinomial', 'penalty':None, 'tol':1e-6, 'max_iter':10000,
        'ag_args': {'name_suffix': 'Base'}},
    {'multi_class':'multinomial', 'penalty':'l2', 'tol':1e-6, 'max_iter':10000, 'C':0.1,
        'ag_args': {'name_suffix': 'Ridge'}},
    {'multi_class':'multinomial', 'penalty':'l1', 'tol':1e-6, 'max_iter':10000, 'C':0.1,
        'ag_args': {'name_suffix': 'Lasso'}},
    {'multi_class':'multinomial', 'penalty':'elasticnet', 'tol':1e-6, 'max_iter':10000, 'C':0.1,
        'ag_args': {'name_suffix': 'ElasticNet'}}
    ]

custom_preset = {'auto_stack': False, 'dynamic_stacking': False,
                'hyperparameters':custom_hyperparameters, 'refit_full': True,
                'set_best_to_refit_full': True, 'save_bag_folds': False,
                'time_limit': time_limit}

In [ ]:
# %%

np.random.seed(2024)

predictor = TabularPredictor(label=label, problem_type='multiclass', eval_metric='log_loss', log_to_file=True)

predictor.fit(train_data, presets=custom_preset)  

No path specified. Models will be saved in: "AutogluonModels\ag-20240824_230546"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          20
Memory Avail:       23.29 GB / 31.75 GB (73.3%)
Disk Space Avail:   772.79 GB / 952.44 GB (81.1%)
Presets specified: [{'auto_stack': False, 'dynamic_stacking': False, 'hyperparameters': {'NN_TORCH': {}, 'GBM': [{'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}, {}, 'GBMLarge'], 'CAT': {}, 'XGB': {}, 'FASTAI': {}, 'RF': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}], 'XT': [{'criterion': 'gin

In [ ]:
# %%

metrics = ['model', 'score_test', 'score_val', 'eval_metric', 'pred_time_test', 'fit_time']

df_leaders = predictor.leaderboard(test_data)

df_leaders.head(40)[metrics]

c:\Users\chiuw\miniconda3\envs\env_AutoGluon_202408\lib\site-packages\sklearn\metrics\_classification.py:2969: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(


,model,score_test,score_val,eval_metric,pred_time_test,fit_time
0,WeightedEnsemble_L2_FULL,-0.136361,NaN,log_loss,0.334878,83.881497
1,WeightedEnsemble_L2,-0.141883,-0.129025,log_loss,0.365770,100.046596
2,CatBoost_FULL,-0.152019,NaN,log_loss,0.036878,56.164296
3,NeuralNetFastAI_FULL,-0.157419,NaN,log_loss,0.080729,12.559018
4,CatBoost,-0.158654,-0.150310,log_loss,0.044850,72.693672
5,XGBoost_FULL,-0.164685,NaN,log_loss,0.128570,1.605628
6,NeuralNetFastAI,-0.165963,-0.166266,log_loss,0.081722,11.950018
7,XGBoost,-0.171801,-0.144754,log_loss,0.180395,4.181003
8,NeuralNetTorch_FULL,-0.175977,NaN,log_loss,0.031893,68.354307
9,LightGBM_FULL,-0.181653,NaN,log_loss,0.075746,0.997662


In [ ]:
# %%

df_leaders.tail(20)[metrics]

,model,score_test,score_val,eval_metric,pred_time_test,fit_time
16,RandomForestGini_FULL,-0.200395,NaN,log_loss,0.184619,1.079046
17,RandomForestGini,-0.208654,-0.206937,log_loss,0.165895,0.956881
18,ExtraTreesEntr_FULL,-0.249184,NaN,log_loss,0.188032,0.766992
19,ExtraTreesGini_FULL,-0.251511,NaN,log_loss,0.214158,0.702703
20,ExtraTreesEntr,-0.260219,-0.267746,log_loss,0.188558,0.712834
21,ExtraTreesGini,-0.263972,-0.268648,log_loss,0.191416,0.701119
22,LightGBMLarge,-0.316343,-0.290976,log_loss,0.080730,4.034518
23,LinearModelLasso_FULL,-0.403137,NaN,log_loss,0.019933,3.305585
24,LinearModelRidge_FULL,-0.403137,NaN,log_loss,0.021926,3.372644
25,LinearModelElasticNet_FULL,-0.403137,NaN,log_loss,0.021927,3.359960


In [ ]:
# %%

y_pred = predictor.predict_proba(test_data.drop(columns=[label]), model='CatBoost_FULL')
y_pred.head()

# predictor.evaluate(test_data, silent=True)

,-12,-10,-8,-6,-4,-2,0,2,4,6,8,10,12
0,4.238544e-05,4.176385e-06,3.812597e-06,2.140584e-03,9.972622e-01,0.000536,7.994241e-07,6.957279e-07,1.225536e-06,5.420325e-06,9.816791e-07,1.186751e-06,7.814633e-07
1,2.851676e-06,3.695846e-06,3.358419e-06,1.007222e-05,2.586520e-04,0.181641,8.159451e-01,2.111561e-03,1.101640e-05,4.627551e-06,2.264560e-06,3.442504e-06,2.837809e-06
2,9.999993e-08,9.999993e-08,9.999993e-08,9.999993e-08,9.999993e-08,0.000062,9.996655e-01,2.711075e-04,2.026826e-07,9.999993e-08,9.999993e-08,9.999993e-08,9.999993e-08
3,3.057247e-07,2.469500e-07,1.852477e-07,8.735338e-07,1.398415e-06,0.000001,1.233405e-07,1.084295e-04,9.944426e-01,5.432843e-03,8.844660e-06,1.211282e-06,1.591582e-06
4,9.999997e-08,9.999997e-08,9.999997e-08,9.999997e-08,9.999997e-08,0.000002,5.238483e-02,9.475742e-01,3.774835e-05,4.100936e-07,1.724376e-07,9.999997e-08,9.999997e-08


In [ ]:
# %%

# "AutogluonModels\ag-20240821_072221"